In [ ]:
!pip install transformers peft accelerate bitsandbytes -U --no-index --find-links /kaggle/input/lmsys-wheel-files

Looking in links: /kaggle/input/lmsys-wheel-files
Processing /kaggle/input/lmsys-wheel-files/peft-0.11.1-py3-none-any.whl
Processing /kaggle/input/lmsys-wheel-files/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl


In [ ]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import (
    Gemma2ForSequenceClassification, GemmaTokenizerFast,
    AutoTokenizer, LlamaForSequenceClassification, BitsAndBytesConfig
)
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel, get_peft_model, LoraConfig, TaskType

torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)

2024-07-29 16:35:23.748325: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 16:35:23.748428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 16:35:23.858682: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Configuration

#### Directory Paths:
 - `gemma_dir`: Directory path for the Gemma-2 model.
 - `gemma_lora_dir`: Directory path for the Gemma LoRA checkpoint. - `llama_model_name`: Directory path for the Llama model.
 - `llama_weights_path`: Directory path for the Llama model weights.

#### Other Parameters:
 - `max_length`: Maximum sequence length.
 - `batch_size`: Batch size for processing.
 - `tta`: Boolean flag for test-time augmentation.
 - `spread_max_length`: Boolean flag for spreading max length.

In [ ]:
@dataclass
class Config:
    gemma_dir: str = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    gemma_lora_dir: str = '/kaggle/input/73zap2gx/checkpoint-5748'
    llama_model_name: str = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
    llama_weights_path: str = '/kaggle/input/lmsys-model/model'
    max_length: int = 2048
    batch_size: int = 4
    tta: bool = False
    spread_max_length: bool = False

# Instantiate the configuration
cfg = Config()

# Load and Process Test Data

#### 🔧 Functions and Operations:
 - `process_text`: A function to process text data by evaluating and joining the text.
 - Apply the `process_text` function to the 'prompt', 'response_a', and 'response_b' columns of the test DataFrame.


In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

# Tokenize Function

 - The function handles different tokenization formats based on the type of tokenizer.
 - It also handles whether to spread the maximum length or not.


In [ ]:
def tokenize(tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length):
    # Handle different formats for different tokenizers
    if isinstance(tokenizer, GemmaTokenizerFast):
        prompt = ["<prompt>: " + p for p in prompt]
        response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
        response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    else:
        prompt = ["User prompt: " + p for p in prompt]
        response_a = ["\n\nModel A :\n" + r_a for r_a in response_a]
        response_b = ["\n\n--------\n\nModel B:\n" + r_b for r_b in response_b]

    # Tokenize with spread max length
    if spread_max_length:
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        attention_mask = [[1] * len(i) for i in input_ids]
    # Tokenize without spread max length
    else:
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask

    return input_ids, attention_mask

# Tokenizer Setup and Data Preparation

In [ ]:
# Gemma Tokenizer
gemma_tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
gemma_tokenizer.add_eos_token = True
gemma_tokenizer.padding_side = "right"

# Llama Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# Prepare data for Gemma model
gemma_data = pd.DataFrame()
gemma_data["id"] = test["id"]
gemma_data["input_ids"], gemma_data["attention_mask"] = tokenize(gemma_tokenizer, test["prompt"], test["response_a"], test["response_b"])
gemma_data["length"] = gemma_data["input_ids"].apply(len)

# Prepare data for Llama model
llama_data = pd.DataFrame()
llama_data["id"] = test["id"]
llama_data["input_ids"], llama_data["attention_mask"] = tokenize(llama_tokenizer, test["prompt"], test["response_a"], test["response_b"])
llama_data["length"] = llama_data["input_ids"].apply(len)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Load Gemma Model on GPU 0


  Apply PEFT using the LoRA checkpoint.

In [ ]:
# Load Gemma model on GPU 0
device_0 = torch.device('cuda:0')
# Load the Gemma model for sequence classification onto GPU 0
gemma_model = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False
)
# Apply PEFT using the LoRA checkpoint
gemma_model = PeftModel.from_pretrained(gemma_model, cfg.gemma_lora_dir)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Load Llama Model on GPU 1

Loads the Llama model for sequence classification onto GPU 1 and applies parameter-efficient fine-tuning (PEFT) using LoRA.

In [ ]:
# Load Llama model on GPU 1
device_1 = torch.device('cuda:1')

# Configure BitsAndBytes for 8-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=False
)
#  Load the Llama model for sequence classification onto GPU 1
llama_base_model = LlamaForSequenceClassification.from_pretrained(
    cfg.llama_model_name,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:1')
llama_base_model.config.pad_token_id = llama_tokenizer.pad_token_id

# Configure and apply PEFT using the LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.10,
    bias='none',
    inference_mode=True,
    task_type=TaskType.SEQ_CLS,
    target_modules=['o_proj', 'v_proj']
)
llama_model = get_peft_model(llama_base_model, peft_config).to(device_1)
llama_model.load_state_dict(torch.load(cfg.llama_weights_path), strict=False)
llama_model.eval()

Unused kwargs: ['bnb_8bit_compute_dtype', 'bnb_8bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/llama-3/transformers/8b-chat-hf/1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
              (v_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1024, bias=False)
            

# Inference Function

 - The function uses mixed precision (automatic casting) for faster and more efficient inference.
 - It uses a no_grad context to avoid computing gradients, which saves memory and computation time.

In [ ]:
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, tokenizer, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []
    # Process the data in batches
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        # Pad the inputs without triggering the fast tokenizer warning
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        # Perform inference
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()

        # Append the probabilities to the corresponding lists
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    # Update the DataFrame with the probabilities
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie

    return df

# Perform Inference Using Threading

Using a ThreadPoolExecutor to run inference on both models in parallel.

In [ ]:
st = time.time()

# Sort data by input length
gemma_data = gemma_data.sort_values("length", ascending=False)
llama_data = llama_data.sort_values("length", ascending=False)

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference,
                           (gemma_data, llama_data),
                           (gemma_model, llama_model),
                           (gemma_tokenizer, llama_tokenizer),
                           (device_0, device_1))

gemma_result_df, llama_result_df = list(results)

# Combine results (simple average)
combined_result_df = gemma_result_df.copy()
combined_result_df["winner_model_a"] = (gemma_result_df["winner_model_a"] + llama_result_df["winner_model_a"]) / 2
combined_result_df["winner_model_b"] = (gemma_result_df["winner_model_b"] + llama_result_df["winner_model_b"]) / 2
combined_result_df["winner_tie"] = (gemma_result_df["winner_tie"] + llama_result_df["winner_tie"]) / 2

print(f"Inference time: {time.time() - st:.2f} seconds")

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Inference time: 6.59 seconds


In [ ]:
submission_df = combined_result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df.head())

,id,winner_model_a,winner_model_b,winner_tie
2,1233961,0.188499,0.534701,0.276801
1,211333,0.350908,0.268337,0.380755
0,136060,0.068339,0.797322,0.134339
